In [10]:
epochs = 10
# We don't use the whole dataset for efficiency purpose, but feel free to increase these numbers
n_train_items = 60032
n_test_items = 10000

# Part X - Secure Training and Evaluation on MNIST

When building Machine Learning as a Service solutions (MLaaS), a company might need to request access to data from other partners to train its model. In health or in finance, both the model and the data are extremely critical: the model parameters is a business asset while data is personal data which is tightly regulated.

In this context, one possible solution is to encrypt both the model and the data and to train the machine learning model over the encrypted values. This guarantees that the company won't access patients medical records for example and that health facilities won't be able to observe the model to which they contribute. Several encryption schemes exist that allow for computation over encrypted data, among which Secure Multi-Party Computation (SMPC), Homomorphic Encryption (FHE/SHE) and Functional Encryption (FE). We will focus here on Multi-Party Computation (which have been introduced in Tutorial 5) which consists of private additive sharing and relies on the crypto protocols SecureNN and SPDZ.

The exact setting of this tutorial is the following: consider that you are the server and you would like to train your model on some data held by $n$ workers. The server secret shares his model and send each share to a worker. The workers also secret share their data and exchange it between them. In the configuration that we will study, there are 2 workers: alice and bob. After exchanging shares, each of them now has one of their own shares, one share of the other worker, and one share of the model. Computation can now start to privately train the model using the appropriate crypto protocols. Once the model is trained, all the shares can be sent back to the server to decrypt it. This is illustrated with the following figure:

![SMPC Illustration](https://github.com/OpenMined/PySyft/raw/11c85a121a1a136e354945686622ab3731246084/examples/tutorials/material/smpc_illustration.png)

To give an example of this process, let's assume alice and bob both hold a part of the MNIST dataset and let's train a model to perform digit classification!

Author:
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel) · GitHub: [@LaRiffle](https://github.com/LaRiffle)

# 1. Encrypted Training demo on MNIST

## Imports and training configuration

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import time

This class describes all the hyper-parameters for the training. Note that they are all public here.

In [12]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = epochs
        self.lr = 0.001
        self.seed = 1
        self.log_interval = 1 # Log info at each batch
        self.precision_fractional = 3

args = Arguments()

_ = torch.manual_seed(args.seed)

Here are PySyft imports. We connect to two remote workers that be call `alice` and `bob` and request another worker called the `crypto_provider` who gives all the crypto primitives we may need.

In [13]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch to add extra functionalities like Federated and Encrypted Learning

# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]
def connect_to_crypto_provider():
    return sy.VirtualWorker(hook, id="crypto_provider")

workers = connect_to_workers(n_workers=2)
crypto_provider = connect_to_crypto_provider()

## Getting access and secret share data

Here we're using a utility function which simulates the following behaviour: we assume the MNIST dataset is distributed in parts each of which is held by one of our workers. The workers then split their data in batches and secret share their data between each others. The final object returned is an iterable on these secret shared batches, that we call the **private data loader**. Note that during the process the local worker (so us) never had access to the data.

We obtain as usual a training and testing private dataset, and both the inputs and labels are secret shared.

In [14]:
def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def one_hot_of(index_tensor):
        """
        Transform to one hot tensor
        
        Example:
            [0, 3, 9]
            =>
            [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
            
        """
        onehot_tensor = torch.zeros(*index_tensor.shape, 10) # 10 classes for MNIST
        onehot_tensor = onehot_tensor.scatter(1, index_tensor.view(-1, 1), 1)
        return onehot_tensor
        
    def secret_share(tensor):
        """
        Transform to fixed precision and secret share a tensor
        """
        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) #mean and std dev on dataset but how do you know if it's secret shared?
    ])
    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=transformation),
        batch_size=args.batch_size
    )
    
    private_train_loader = [
        (secret_share(data), secret_share(one_hot_of(target)))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transformation),
        batch_size=args.test_batch_size
    )
    
    private_test_loader = [
        (secret_share(data), secret_share(target.float()))
        for i, (data, target) in enumerate(test_loader)
        if i < n_test_items / args.test_batch_size
    ]
    
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=args.precision_fractional,
    workers=workers,
    crypto_provider=crypto_provider
)

## Model specification

Here is the model that we will use, it's a rather simple one but [it has proved to perform reasonably well on MNIST](https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627)

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

## Training and testing functions

The training is done almost as usual, the real difference is that we can't use losses like negative log-likelihood (`F.nll_loss` in PyTorch) because it's quite complicated to reproduce these functions with SMPC. Instead, we use a simpler Mean Square Error loss.

In [16]:
def train(args, model, private_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(private_train_loader): # <-- now it is a private dataset
        start_time = time.time()
        
        optimizer.zero_grad()
        
        output = model(data)
        
        # loss = F.nll_loss(output, target)  <-- not possible here
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            loss = loss.get().float_precision()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.3f}s'.format(
                epoch, batch_idx * args.batch_size, len(private_train_loader) * args.batch_size,
                100. * batch_idx / len(private_train_loader), loss.item(), time.time() - start_time))
            

The test function does not change!

In [17]:
def test(args, model, private_test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in private_test_loader:
            start_time = time.time()
            
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum()

    correct = correct.get().float_precision()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct.item(), len(private_test_loader)* args.test_batch_size,
        100. * correct.item() / (len(private_test_loader) * args.test_batch_size)))

### Let's launch the training !

A few notes about what's happening here. First, we secret share all the model parameters across our workers. Second, we convert optimizer's hyperparameters to fixed precision. Note that we don't need to secret share them because they are public in our context, but as secret shared values live in finite fields we still need to move them in finite fields using `.fix_precision`, in order to perform consistently operations like the weight update $W \leftarrow W - \alpha * \Delta W$.

In [18]:
model = Net()
model = model.fix_precision().share(*workers, crypto_provider=crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
optimizer = optimizer.fix_precision() 

for epoch in range(1, args.epochs + 1):
    train(args, model, private_train_loader, optimizer, epoch)
    test(args, model, private_test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 1.591000	Time: 19.713s
Train Epoch: 1 [64/60032 (0%)]	Loss: 1.480000	Time: 19.417s
Train Epoch: 1 [128/60032 (0%)]	Loss: 1.251000	Time: 17.217s
Train Epoch: 1 [192/60032 (0%)]	Loss: 1.377000	Time: 17.683s
Train Epoch: 1 [256/60032 (0%)]	Loss: 1.148000	Time: 18.011s
Train Epoch: 1 [320/60032 (1%)]	Loss: 1.074000	Time: 16.910s
Train Epoch: 1 [384/60032 (1%)]	Loss: 1.142000	Time: 18.051s
Train Epoch: 1 [448/60032 (1%)]	Loss: 1.193000	Time: 16.061s
Train Epoch: 1 [512/60032 (1%)]	Loss: 1.195000	Time: 16.852s
Train Epoch: 1 [576/60032 (1%)]	Loss: 1.193000	Time: 16.225s
Train Epoch: 1 [640/60032 (1%)]	Loss: 1.067000	Time: 16.616s
Train Epoch: 1 [704/60032 (1%)]	Loss: 1.016000	Time: 23.608s
Train Epoch: 1 [768/60032 (1%)]	Loss: 0.975000	Time: 16.591s
Train Epoch: 1 [832/60032 (1%)]	Loss: 1.106000	Time: 16.995s
Train Epoch: 1 [896/60032 (1%)]	Loss: 0.996000	Time: 16.584s
Train Epoch: 1 [960/60032 (2%)]	Loss: 1.075000	Time: 16.949s
Train Epoch: 1 [1024/60032 

Train Epoch: 1 [8448/60032 (14%)]	Loss: 0.617000	Time: 22.833s
Train Epoch: 1 [8512/60032 (14%)]	Loss: 0.554000	Time: 25.748s
Train Epoch: 1 [8576/60032 (14%)]	Loss: 0.596000	Time: 19.574s
Train Epoch: 1 [8640/60032 (14%)]	Loss: 0.711000	Time: 22.589s
Train Epoch: 1 [8704/60032 (14%)]	Loss: 0.710000	Time: 24.045s
Train Epoch: 1 [8768/60032 (15%)]	Loss: 0.653000	Time: 24.684s
Train Epoch: 1 [8832/60032 (15%)]	Loss: 0.794000	Time: 24.314s
Train Epoch: 1 [8896/60032 (15%)]	Loss: 0.597000	Time: 19.799s
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.567000	Time: 20.128s
Train Epoch: 1 [9024/60032 (15%)]	Loss: 0.566000	Time: 19.261s
Train Epoch: 1 [9088/60032 (15%)]	Loss: 0.661000	Time: 20.037s
Train Epoch: 1 [9152/60032 (15%)]	Loss: 0.580000	Time: 19.384s
Train Epoch: 1 [9216/60032 (15%)]	Loss: 0.632000	Time: 19.982s
Train Epoch: 1 [9280/60032 (15%)]	Loss: 0.637000	Time: 20.266s
Train Epoch: 1 [9344/60032 (16%)]	Loss: 0.590000	Time: 20.159s
Train Epoch: 1 [9408/60032 (16%)]	Loss: 0.617000	Time: 

Train Epoch: 1 [16704/60032 (28%)]	Loss: 0.570000	Time: 24.444s
Train Epoch: 1 [16768/60032 (28%)]	Loss: 0.635000	Time: 24.536s
Train Epoch: 1 [16832/60032 (28%)]	Loss: 0.595000	Time: 24.665s
Train Epoch: 1 [16896/60032 (28%)]	Loss: 0.563000	Time: 24.690s
Train Epoch: 1 [16960/60032 (28%)]	Loss: 0.575000	Time: 25.739s
Train Epoch: 1 [17024/60032 (28%)]	Loss: 0.547000	Time: 24.176s
Train Epoch: 1 [17088/60032 (28%)]	Loss: 0.594000	Time: 25.234s
Train Epoch: 1 [17152/60032 (29%)]	Loss: 0.617000	Time: 29.819s
Train Epoch: 1 [17216/60032 (29%)]	Loss: 0.533000	Time: 25.418s
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.457000	Time: 24.750s
Train Epoch: 1 [17344/60032 (29%)]	Loss: 0.564000	Time: 24.985s
Train Epoch: 1 [17408/60032 (29%)]	Loss: 0.485000	Time: 24.844s
Train Epoch: 1 [17472/60032 (29%)]	Loss: 0.629000	Time: 24.398s
Train Epoch: 1 [17536/60032 (29%)]	Loss: 0.669000	Time: 24.477s
Train Epoch: 1 [17600/60032 (29%)]	Loss: 0.558000	Time: 23.896s
Train Epoch: 1 [17664/60032 (29%)]	Loss:

Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.487000	Time: 22.515s
Train Epoch: 1 [25024/60032 (42%)]	Loss: 0.398000	Time: 26.853s
Train Epoch: 1 [25088/60032 (42%)]	Loss: 0.466000	Time: 24.606s
Train Epoch: 1 [25152/60032 (42%)]	Loss: 0.475000	Time: 22.632s
Train Epoch: 1 [25216/60032 (42%)]	Loss: 0.435000	Time: 23.915s
Train Epoch: 1 [25280/60032 (42%)]	Loss: 0.556000	Time: 23.143s


Exception ignored in: <bound method ObjectPointer.__del__ of [PointerTensor | me:82219469804 -> worker1:77050876030]>
Traceback (most recent call last):
  File "F:\Anaconda3\envs\tfe\lib\site-packages\syft\generic\pointers\object_pointer.py", line 344, in __del__
    self.owner.send_msg(ForceObjectDeleteMessage(self.id_at_location), self.location)
  File "F:\Anaconda3\envs\tfe\lib\site-packages\syft\workers\base.py", line 279, in send_msg
    bin_message = sy.serde.serialize(message, worker=self)
  File "F:\Anaconda3\envs\tfe\lib\site-packages\syft\serde\serde.py", line 43, in serialize
    return strategy(obj, worker, simplified, force_full_simplification)
  File "F:\Anaconda3\envs\tfe\lib\site-packages\syft\serde\msgpack\serde.py", line 329, in serialize
    return _serialize_msgpack_binary(simple_objects)
  File "F:\Anaconda3\envs\tfe\lib\site-packages\syft\serde\msgpack\serde.py", line 283, in _serialize_msgpack_binary
    binary = msgpack_lib.dumps(simple_objects)
  File "F:\Anaco

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 15872000 bytes. Buy new RAM!


There you are! You just get 75% of accuracy using a tiny fraction of the MNIST dataset, using 100% encrypted training!

# 2. Discussion

Let's have a closer look to the power of encrypted training by analyzing what we just did.

## 2.1 Computation time

First thing is obviously the running time! As you have surely noticed, it is far slower than plain text training. In particular, a iteration over 1 batch of 64 items takes 3.2s while only 13ms in pure PyTorch. Whereas this might seem like a blocker, just recall that here everything happened remotely and in the encrypted world: no single data item has been disclosed. More specifically, the time to process one item is 50ms which is not that bad. The real question is to analyze when encrypted training is needed and when only encrypted prediction is sufficient. 50ms to perform a prediction is completely acceptable in a production-ready scenario for example!

One main bottleneck is the use of costly activation functions: relu activation with SMPC are very costly because it uses private comparison and the SecureNN protocol. As an illustration, if we replace relu with a quadratic activation as it is done in several papers on encrypted computation like CryptoNets, we drop from 3.2s to 1.2s.

As a general rule, the key idea to take away is to encrypt only what's necessary, and this tutorial shows you how simple it can be.

## 2.2 Backpropagation with SMPC

You might wonder how we perform backpropagation and gradient updates although we're working with integers in finite fields. To do so, we have developed a new syft tensor called AutogradTensor. This tutorial used it intensively although you might have not seen it! Let's check this by printing a model's weight:

In [10]:
model.fc3.bias

Parameter containing:
(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:75794498061 -> worker1:82412848383]
	-> [PointerTensor | me:46385483913 -> worker2:11286612429]
	*crypto provider: crypto_provider*

And a data item:

In [11]:
first_batch, input_data = 0, 0
private_train_loader[first_batch][input_data]

(Wrapper)>AutogradTensor>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:32162462403 -> worker1:43976326703]
	-> [PointerTensor | me:6529537735 -> worker2:65226254314]
	*crypto provider: crypto_provider*

As you observe, the AutogradTensor is there! It lives between the torch wrapper and the FixedPrecisionTensor which indicate that the values are now in finite fields. The goal of this AutogradTensor is to store the computation graph when operations are made on encrypted values. This is useful because when calling backward for the backpropagation, this AutogradTensor overrides all the backward functions that are not compatible with encrypted computation and indicates how to compute these gradients. For example, regarding multiplication which is done using the Beaver triples trick, we don't want to differentiate the trick all the more that differentiating a multiplication should be very easy: $\partial_b (a \cdot b) = a \cdot \partial b$. Here is how we describe how to compute these gradients for example:

```python
class MulBackward(GradFunc):
    def __init__(self, self_, other):
        super().__init__(self, self_, other)
        self.self_ = self_
        self.other = other

    def gradient(self, grad):
        grad_self_ = grad * self.other
        grad_other = grad * self.self_ if type(self.self_) == type(self.other) else None
        return (grad_self_, grad_other)
```

You can have a look at `tensors/interpreters/gradients.py` if you're curious to see how we implemented more gradients.

In terms of computation graph, it means that a copy of the graph remains local and that the server which coordinates the forward pass also provide instructions on how to do the backward pass. This is a completely valid hypothesis in our setting.

## 2.3 Security guarantees


Last, let's give a few hints about the security we're achieving here: adversaries that we are considering here are **honest but curious**: this means that an adversary can't learn anything about the data by running this protocol, but a malicious adversary could still deviate from the protocol and for example try to corrupt the shares to sabotage the computation. Security against malicious adversaries in such SMPC computations including private comparison is still an open problem.

In addition, even if Secure Multi-Party Computation ensures that training data wasn't accessed, many threats from the plain text world are still present here. For example, as you can make request to the model (in the context of MLaaS), you can get predictions which might disclose information about the training dataset. In particular you don't have any protection against membership attacks, a common attack on machine learning services where the adversary wants to determine if a specific item was used in the dataset. Besides this, other attacks such as unintended memorization processes (models learning specific feature about a data item), model inversion or extraction are still possible.

One general solution which is effective for many of the threats mentioned above is to add Differential Privacy. It can be nicely combined with Secure Multi-Party Computation and can provide very interesting security guarantees. We're currently working on several implementations and hope to propose an example that combines both shortly!

# Conclusion

As you have seen, training a model using SMPC is not complicated from a code point of view, even we use rather complex objects under the hood. With this in mind, you should now analyse your use-cases to see when encrypted computation is needed either for training or for evaluation. If encrypted computation is much slower in general, it can also be used carefully so as to reduce the overall computation overhead.

If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways! 

### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)

In [ ]:
# speed without encyption locally
model = Net()
print(model)

optimizer = optim.SGD(model.parameters(), lr=args.lr)

transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) #mean and std dev on dataset but how do you know if it's secret shared?
    ])

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=transformation),
        #batch_size=args.batch_size
    )
    

    
for epoch in range(1, args.epochs + 1):
    running_loss = 0
    for images, labels in train_loader:
        log_ps = model(images)
        loss = ((log_ps - labels)**2).sum()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(train_loader)}")
        
        
        #test(args, model, private_test_loader)

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)
